In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import folium
from amount_sum import *

# Amount of money received per canton

The functions used in this notebook are implemented in amount_sum.py file.

## Load P3 data

In [2]:
# load cantons dictionnary from JSON file
with open('data/cantons.json', 'r') as ff:
    cantons = json.load(ff)
    
# load df DataFrame of P3 database
df = pd.read_csv('data/P3_GrantExport.csv', header=0, sep=';')
df = df.fillna('')

## Clean P3 data

In [3]:
# Remove the `fail` entry
del cantons['fail']

# 'Approved Amount' field of DF is of type STRING because of its custom NA value stated as "data not included in P3"
# We need to change this value to be able, aftward, to states 'Approved Amount' field as a FLOAT type

# As the objectiv is to calculate the total amount assigned to each canton, the unknown amount will simply be put to 0
df.loc[df['Approved Amount'] == 'data not included in P3', 'Approved Amount'] = 0
df['Approved Amount']= df['Approved Amount'].astype(float)


## Assign total amount assigned to each canton

In [4]:
from collections import defaultdict

# create a dictionary with canton name and the assigned amount
canton_money = dict()
for i in cantons:
    money = sum_by_canton(df, cantons, i)
    canton_money[i] = money

# deal with missing canton (ie the ones that doesn't receive any money)
# default dictionnary
canton_money = defaultdict(lambda: 0, canton_money)


# Map Vizualization

** Transforme ch-canton.json from TopoJson to GeoJson** using [jeffpaine.github.io/geojson-topojson](http://jeffpaine.github.io/geojson-topojson/)

In [5]:
# load GeoJson of swiss canton
canton_topo = json.load(open('data/ch-canton.geojson.json'))

In [6]:
import folium.colormap as cm

# prepare color line for canton between min and max amount of money
min_money = min(canton_money.items(), key=lambda a: a[1])[1]
max_money = max(canton_money.items(), key=lambda a: a[1])[1]

linear = cm.LinearColormap(['yellow','yellowgreen', 'green', 'SeaGreen'],
                           vmin= int(min_money), vmax= int(max_money))
linear

In [7]:
m = folium.Map([46.8,8.3], tiles='cartodbpositron', zoom_start=8)

folium.GeoJson(
    canton_topo,
        style_function=lambda feature: {
        'fillColor': linear(canton_money[feature['id']]),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

m.save(os.path.join('results', 'Colormaps_0.html'))

m